In [1]:
import json
import urllib.request
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap
import seaborn as sns

# Filtering stations on common stations in main dataset and dataset of bike station capacity

Because not all the stations in the main dataset is registered with a capacity in the dataset with station information, only the common stations needs to be used for the analysis.

### Import and convert dataset with station info from json objects to a pandas dataframe. 

In [2]:
#Import data from json url. Delete unnecessary columns
with urllib.request.urlopen("https://feeds.citibikenyc.com/stations/stations.json") as url:
    data = json.loads(url.read().decode())
    del data['executionTime']
    data = data['stationBeanList']

#Add data from json objects to list. Only include necessary fields
stations = []
for station in data:
    stations.append([station['id'], station['stationName'], station['latitude'], station['longitude'], station['totalDocks']])

#make dataframe of station info from list derived from json object
df1 = pd.DataFrame(stations, columns=["id", "stationname", "lat", "long", "capacity"])

In [3]:
df1.head()

,id,stationname,lat,long,capacity
0,168,W 18 St & 6 Ave,40.739713,-73.994564,47
1,281,Grand Army Plaza & Central Park S,40.764397,-73.973715,66
2,285,Broadway & E 14 St,40.734546,-73.990741,53
3,304,Broadway & Battery Pl,40.704633,-74.013617,33
4,324,DeKalb Ave & Hudson Ave,40.689888,-73.981013,51


### Import all trip data from 2016-2018

In [4]:
def getDataByYear(year):
    name=""
    df=[]
    
    for k in range(1,13):
        if k<10:
            name = str(year)+"0"+str(k)
        else:
            name = str(year)+""+str(k)
        name +="-citibike-tripdata.csv"
        
        df.append(pd.read_csv('data/'+name))
        #set columns to lower case
        df[k-1].columns = map(str.lower, df[k-1].columns)
        #
        df[k-1].columns = [x.replace(" ","") for x in list(df[k-1].columns)]
        df[k-1]["starttime"] = pd.to_datetime(df[k-1]["starttime"], infer_datetime_format=True)
        df[k-1]["stoptime"] = pd.to_datetime(df[k-1]["stoptime"], infer_datetime_format=True)
        print(name)

    return pd.concat(df,ignore_index=True)

In [5]:
df16 = getDataByYear(2016)

201601-citibike-tripdata.csv
201602-citibike-tripdata.csv
201603-citibike-tripdata.csv
201604-citibike-tripdata.csv
201605-citibike-tripdata.csv
201606-citibike-tripdata.csv
201607-citibike-tripdata.csv
201608-citibike-tripdata.csv
201609-citibike-tripdata.csv
201610-citibike-tripdata.csv
201611-citibike-tripdata.csv
201612-citibike-tripdata.csv


In [6]:
df17 = getDataByYear(2017)

201701-citibike-tripdata.csv
201702-citibike-tripdata.csv
201703-citibike-tripdata.csv
201704-citibike-tripdata.csv
201705-citibike-tripdata.csv
201706-citibike-tripdata.csv
201707-citibike-tripdata.csv
201708-citibike-tripdata.csv
201709-citibike-tripdata.csv
201710-citibike-tripdata.csv
201711-citibike-tripdata.csv
201712-citibike-tripdata.csv


In [7]:
df18 = getDataByYear(2018)

201801-citibike-tripdata.csv
201802-citibike-tripdata.csv
201803-citibike-tripdata.csv
201804-citibike-tripdata.csv
201805-citibike-tripdata.csv
201806-citibike-tripdata.csv
201807-citibike-tripdata.csv
201808-citibike-tripdata.csv
201809-citibike-tripdata.csv
201810-citibike-tripdata.csv
201811-citibike-tripdata.csv
201812-citibike-tripdata.csv


### Delete unnecessary columns and change to datetime objects

In [8]:
frames = [df16,df17,df18]
df=pd.concat(frames)
del_cols = ['tripduration', 'startstationname', 'startstationlatitude', 'startstationlongitude', 'endstationname',
       'endstationlatitude', 'endstationlongitude', 'usertype', 'birthyear', 'gender', 'bikeid']
df = df.drop(del_cols, axis=1)
df['startday'] = df['starttime'].dt.dayofweek
df['stopday'] = df['stoptime'].dt.dayofweek
df['starttime'] = df['starttime'].dt.hour
df['stoptime'] = df['stoptime'].dt.hour

In [9]:
df.head()

,starttime,stoptime,startstationid,endstationid,startday,stopday
0,0,0,268.0,3002.0,4,4
1,0,0,476.0,498.0,4,4
2,0,0,489.0,284.0,4,4
3,0,0,268.0,3002.0,4,4
4,0,0,2006.0,2006.0,4,4


### Find stations that have both incoming and outgoing bikes

In [11]:
#Find stations that have both incoming and outgoing bikes
df2 = df[['startstationid']].drop_duplicates()
df3 = df[['endstationid']].drop_duplicates()
df4 = pd.merge(df2,df3, how='inner', left_on='startstationid', right_on='endstationid')
df4 = df4[['startstationid']].drop_duplicates().dropna()
df4['startstationid'] = df4['startstationid'].apply(lambda x : int(x))
df4.columns = ['id']

In [12]:
df4.head()

,id
0,268
1,476
2,489
3,2006
4,3104


### Find stations that are in both dataset, and merge these

In [13]:
df_common_stations = pd.merge(df1, df4, how='inner', on=['id'])
df_common_stations.to_csv('common_stations.csv', index=False)
df_common_stations = df_common_stations.sort_values(by='id')

In [14]:
df_common_stations.head()

,id,stationname,lat,long,capacity
18,72,W 52 St & 11 Ave,40.767272,-73.993929,55
19,79,Franklin St & W Broadway,40.719116,-74.006667,33
20,82,St James Pl & Pearl St,40.711174,-74.000165,27
21,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,62
22,119,Park Ave & St Edwards St,40.696089,-73.978034,19


### Count bikes in and out for weekdays and hours

In [ ]:
df_start = df[['startstationid', 'startday','starttime']]
df_stop = df[['endstationid', 'stopday', 'stoptime']]
df_start= pd.DataFrame({'startcount' : df_start.groupby(['startstationid', 'startday', 'starttime']).size()}).reset_index()
df_stop = pd.DataFrame({'stopcount' : df_stop.groupby(['endstationid', 'stopday', 'stoptime']).size()}).reset_index()

In [16]:
df_start.head()

,startstationid,startday,starttime,startcount
0,72.0,0,0,59
1,72.0,0,1,54
2,72.0,0,2,33
3,72.0,0,3,20
4,72.0,0,4,47


In [17]:
df_stop.head()

,endstationid,stopday,stoptime,stopcount
0,72.0,0,0,89
1,72.0,0,1,46
2,72.0,0,2,23
3,72.0,0,3,12
4,72.0,0,4,18


### Merge in and out data

In [23]:
df_count = pd.merge(df_start, df_stop, left_on=['startstationid', 'startday', 'starttime'], 
                    right_on=['endstationid', 'stopday', 'stoptime'])
df_count = df_count.drop(['endstationid', 'stopday', 'stoptime'], axis=1)
df_count = df_count.rename(columns={'startstationid': 'id', 'startday': 'weekday','starttime': 'hour'})

### Calculate netcount and demand

In [24]:
df_count['netcount'] = df_count['stopcount'] - df_count['startcount']
#df_count = pd.merge(df_count,df_common_stations[['id','capacity']],on='id', how='left')
df_count = pd.merge(df_count, df1[['id','capacity']],on='id', how='left')
#drop rows with stations that are not in capacity dataset
df_count = df_count[np.isfinite(df_count['capacity'])]

#Demand function
df_count['demand'] = -(df_count['netcount'])/df_count['capacity']

df_count['id'] = df_count['id'].apply(lambda x : int(x))

#drop rows with stations that have 0 in capacity
df_count = df_count[df_count.capacity != 0]
#drop duplicates
df_count = df_count.drop_duplicates() 

In [25]:
df_count.head()

,id,weekday,hour,startcount,stopcount,netcount,capacity,demand
0,72,0,0,59,89,30,55.0,-0.545455
1,72,0,1,54,46,-8,55.0,0.145455
2,72,0,2,33,23,-10,55.0,0.181818
3,72,0,3,20,12,-8,55.0,0.145455
4,72,0,4,47,18,-29,55.0,0.527273


### Save filtered data to csv

In [26]:
#make csv of capacity
df_count.to_csv('capacity1.csv', index=False)